# 2-step Diagnosis Pipeline

Import relevant functions

In [1]:
import os
import json
from glob import glob
import AIplus_Utilities as PU

import torch
import torchmetrics as tm

from PIL import Image
from PIL.ImageOps import grayscale
from numpy import mean
from sklearn.metrics import confusion_matrix

global device
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
print(device)

cpu


Confirm CPU environment for YOLO

In [2]:
import ultralytics
from ultralytics import YOLO
print(ultralytics.checks())


Ultralytics YOLOv8.0.142  Python-3.10.2 torch-2.0.1+cpu CPU (11th Gen Intel Core(TM) i7-1185G7 3.00GHz)
Setup complete  (8 CPUs, 15.7 GB RAM, 408.0/952.6 GB disk)


None


# Prediction Pipeline - Simple

In [4]:
# Set path to data
this_path = os.getcwd()
#data_basepath = label_basepath = this_path+'\\Data\\Ultrasound-labeled\\'
#data_basepath = label_basepath = this_path+'\\Data\\External_Test_set\\'
#data_basepath = label_basepath = this_path+'\\Data\\Simple_Test_Images\\'
#data_basepath = label_basepath = this_path+'\\Data\\AIplus_validation\\'
data_basepath = label_basepath = this_path+'\\Data\\External_test_cropped\\'
model_basepath = this_path+'\\Trained_Models\\'

# Set flag for detailed narrative output
#verbose = 2    # full narrative plus labeled images, best for small dataset troubleshooting
#verbose = True # narrative only
verbose = False # minimum output narrative

# Load segmentation model
trained_seg_model = YOLO(model_basepath+'best_yolo_seg3.pt')    # trained on un-enhanced cropped AI+ and public datasets

# Load classification model
trained_class_model = torch.load(model_basepath+'best_densenet_6.pth',map_location=torch.device(device))

# Get image names and provided labels from directories above
print('Loading images and labels')
label_df, label_list = PU.load_aiplus_labels(label_basepath)
image_df, image_list = PU.load_aiplus_images(data_basepath,label_list)

# Predict labels using segmentation model
print('Predicting label segmentation boxes')
pred_label_df = PU.run_seg_model(trained_seg_model,list(image_df['image_path']),verbose=verbose)
pred_label_df['label_bool'] = label_df['label_bool']
pred_label_df['image_path'] = label_df['image_path']

# Create Dataloader for test images, using dataframe and provided labels
print('Creating Dataloader')
pred_dl = PU.make_dataloader(pred_label_df,data_basepath,verbose=verbose)

# Predict labels from classification model
print('Predicting diagnosis class labels')
preds, true_y = PU.predict_classes(pred_dl,trained_class_model,device,verbose=verbose)
p = torch.Tensor.tolist(preds)
ty = torch.Tensor.tolist(true_y)
pred_label_df['label_bool'] = [int(pi) for pi in p]

# Evaluate DSC score for seg model bounding boxes against provided labels
pred_label_scores = PU.score_labels_dsc(label_df,pred_label_df,verbose=verbose)
print('Overall DSC score for bounding box segmentation:',round(mean(pred_label_scores*100),3))
print('     Based on',len(pred_label_scores),'images')

# Return metrics
metrics = PU.score_classes(preds,true_y)
map_scores = PU.map_scores(PU.label_df_to_dict(label_df),PU.label_df_to_dict(pred_label_df))
print('Overall metrics for diagnosis:')
print(metrics)
print(map_scores)

# Predict BI-RADS ratings
# pending

Loading images and labels
Predicting label segmentation boxes



0: 160x160 1 tumor, 1: 160x160 1 tumor, 2: 160x160 2 tumors, 3: 160x160 1 tumor, 95.4ms
Speed: 1.3ms preprocess, 23.8ms inference, 3.1ms postprocess per image at shape (1, 3, 160, 160)


Segmentation error count: 0
Creating Dataloader
Predicting diagnosis class labels
# batches / inputs: 1 4
Overall DSC score for bounding box segmentation: 0.78
     Based on 4 images
Overall metrics for diagnosis:
{'Accuracy': 0.25, 'Precision': 1.0, 'Recall': 0.25, 'F1': 0.4, 'ConfMat': tensor([[0, 0],
        [3, 1]])}
[0.25, 0.025]
